In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rasterio
import os
import cv2
import keras 
from keras.utils import normalize
from keras import *
import pandas as pd


# Data preperation

## Read the points and convert it to polygons

In [ ]:
# Open the area which we need to map its flood susceptibility as a point
# We are just repeating the same steps as reading the training data
points=gpd.read_file('Points.shp')
points.plot()

In [ ]:
points['index_ID']=points.index

In [ ]:
buffer_dist=115 #buffer distance = image size x spatial resolution /2

# Read in the shapefile
points = gpd.read_file("Points.shp")

# Create square buffers with a side length of buffer_dist units around the point features
points['geometry'] = points.buffer(buffer_dist)

points['geometry'] = points.geometry.envelope

# Save the new shapefile
#points.to_file("squares.shp")

In [ ]:
# Iterate over each feature in the shapefile
for index, feature in points.iterrows():
    # Create a new GeoDataFrame with just the current feature
    #print(index)
    
    feature_gdf = points.iloc[[index]]
    #print(feature_gdf)
    #print(feature_gdf['Label'][index])
    
    
        
    # Save the feature to a new shapefile
    feature_gdf.to_file(r"D:\USER\seleem\Omar_CNN\Tag_der_hydrologie\Git_hub\divided_mapping\feature_{}.shp".format(index))

 ## Open the predictive feaure raster and create images for each feature

In [ ]:
# Read raster files with GDAL
# import 
ds = gdal.Open("Composite_raster.tif") # open a raster with several bands, each band  represent one predictive feature
gt= ds.GetGeoTransform() #get the transformation data
proj = ds.GetProjection() #get the projection

band = ds.GetRasterBand(1) #read the first band 
array = band.ReadAsArray() #read the first band as an array

plt.figure()  #plot the raster to check that you every thing is working well
plt.imshow(array)

In [ ]:
# change the path to the folder where we saved the splitted polygons
shp_path=r"D:\USER\seleem\Omar_CNN\Tag_der_hydrologie\Git_hub\divided_mapping"
os.chdir(shp_path)
shp_file = glob.glob('*.shp')
#index =0
for file in shp_file:
        #print(str(file))         
        ds2 = ogr.Open(file, 1)
        layer = ds2.GetLayer()
        shp_ds=gpd.read_file(file)
        #print(shp_ds['Label'][0])
        #index+=1
        # we will clip the raster with each polygon and save the flooded and notflooded locations in different folders
        # we will check the label, if label =0 then this is not flooded location
        #if shp_ds['Label'][0] == 0 : # clip and save not flooded locations
             #Save the feature to a new shapefile
            #dsClip = gdal.Warp(r"D:\USER\seleem\Omar_CNN\Tag_der_hydrologie\Git_hub\Predictive_features\NotFlooded\feature_"+str(file[:-4])+".tif", ds, cutlineDSName = file,
                       #cropToCutline = True, dstNodata = np.nan)
        #else: # clip and save flooded locations
            # Save the feature to a new shapefile
                  
        dsClip = gdal.Warp(r"D:\USER\seleem\Omar_CNN\Tag_der_hydrologie\Git_hub\Mapping"+str(file[:-4])+".tif", ds, cutlineDSName = file,
                       cropToCutline = True, dstNodata = np.nan)

## Read the images

In [ ]:
data_path=r"D:\USER\seleem\Omar_CNN\Tag_der_hydrologie\Git_hub\Mapping" # created in data_preperation script


In [ ]:
# Read the DEM and save it in a list

DEM=[]
Slope=[]
TWI=[]
DTRoad=[]
DTRiver=[]
CN=[]
Rain=[]
Aspect=[]
Curve=[]
Freq=[]
DTDrainage=[]
name=[]
# we have 11 predictive features 
# every feature is presented in one band 
# loop over the predictive features
predictive_features=[DEM, Slope, TWI, DTRoad, DTRiver, CN, Rain, Aspect, Curve, Freq, DTDrainage] # the features muss be in the same order as the bands in the composite raster

print(len(predictive_features))
    


In [ ]:
def create_training_data():
    for i in range(len(predictive_features)):
        print(i+1)
        #for category in CATEGORIES:
        path= os.path.join(data_path) # Path to flooded and NotFlooded dir
        #class_num= CATEGORIES.index(category)
            for img in os.listdir(path):
                try:
                    img_open=rasterio.open(os.path.join(path,img))
                    #print(category,img,class_num)
                    img_array=img_open.read(i+1)
                    #img_array= np.where(img_array < 0, np.nan,img_array)
                    #mean=np.nanmean(img_array)
                    #img_array= np.where(np.isnan(img_array), mean,img_array)
                    #new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize to normalize data size
                    predictive_features[i].append(img_array)
                    
                    if i==0:
                        name.append(img)
                        #print(class_num)
                except Exception as e:
                    pass
create_training_data()

In [ ]:
# convert the predictive feature lists to numpy arrays
DEM_array=np.array(DEM).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Slope_array=np.array(Slope).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
TWI_array=np.array(TWI).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
DTRoad_array=np.array(DTRoad).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
DTRiver_array=np.array(DTRiver).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
CN_array=np.array(CN).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
DTDrainage_array=np.array(DTDrainage).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Aspect_array=np.array(Aspect).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Curvature_array=np.array(Curve).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Freq_Curve_array=np.array(Freq).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
Rain_array=np.array(Rain).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

In [ ]:
# concatenate all the predicvtive feature arrays into one array
X_array=np.concatenate([DEM_array,Slope_array,TWI_array,DTRoad_array, DTRiver_array,CN_array,Rain_array,Aspect_array, Curvature_array, Freq_Curve_array, DTDrainage_array], axis=-1)

## Prediction and mapping

In [ ]:
from keras.models import load_model
model = load_model('LeNet_11_inputs_23_imgsize_normalized_input_rasters_python.h5')


In [ ]:
Y_predict=model.predict(X_array)

In [ ]:
df_predict=pd.DataFrame()
df_predict['FSM']=Y_predict
df_predict['name']=name
df_predict['index_ID']=df_predict.name.str[8:-4].astype(int)

In [ ]:
FSM_df=pd.merge(points,df_predict,on="index_ID")
FSM_df

In [ ]:
FSM_df.to_file("FSM.shp") # convert it to a raster in QGIS or arcmap or in python


In [ ]:
# Converting the point shapefile to raster.
# We will use the model prediction (column FSM in df_SA to make a raster)
from geocube.api.core import make_geocube
import rasterio as rio

out_grid= make_geocube(vector_data=FSM_df, measurements=["FSM"], resolution=(-1, 1)) #for most crs negative comes first in resolution
out_grid["FSM"].rio.to_raster("Flood_susceptibility.tif")